In [1]:
import os
import utils
import preprocess
import encoders
import datastore
import yaml

08/14/2021 15:02:53 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
08/14/2021 15:02:54 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: paraphrase-mpnet-base-v2
08/14/2021 15:02:55 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cpu
08/14/2021 15:03:03 - INFO - sentence_transformers.cross_encoder.CrossEncoder -   Use pytorch device: cpu


In [2]:
with open(r"C:\Users\tanch\Documents\Coding Competitions\SMU\SMU HACKATHON 2021\config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)

{'index': 'covid_datastore', 'bi_encoder': 'paraphrase-mpnet-base-v2', 'cross_encoder': 'cross-encoder/ms-marco-MiniLM-L-6-v2', 'vector_similarity': 'cosine', 'directory': 'C:\\Users\\tanch\\Documents\\Coding Competitions\\SMU\\SMU HACKATHON 2021\\Long-John-Silvers---SMU-Hackathon-2021\\data'}


# query()
- returns k most similar passages(sorted), given a query

In [3]:
from datetime import date
import pandas as pd
def query(query: str,  top_k = 3, max_words = 30):
    today = date.today()
    date_range = list(pd.date_range(start=today,periods=100))
    query_embedding = encoders.bi_encoder.encode(query, show_progress_bar = False)
    hits = datastore.datastore.query_by_embedding(query_embedding,top_k = 10,filters = {"end_date":date_range})   # bi encoder retrieve
    shortened_hits = []
    for hit in hits:
        shortened_hits.extend(preprocess.split_document(hit,max_words))
    for hit in shortened_hits:
        hit.score = encoders.cross_encoder.predict([query,hit.text], show_progress_bar = False)                                          # cross encoder rerank
    return sorted(hits,key = lambda x: x.score, reverse = True)[:top_k]

In [4]:
query("sick employees", top_k = 3, max_words = 30)

C:\Users\tanch\anaconda3\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
08/14/2021 15:03:03 - INFO - elasticsearch -   POST http://localhost:9200/covid_datastore/_search [status:200 request:0.039s]
08/14/2021 15:03:03 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
08/14/2021 15:03:03 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
08/14/2021 15:03:03 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.
08/14/2021 15:03:03 - WARNING - haystack.preprocesso

[{'text': 'Any employee who is feeling unwell or showing symptoms of illness should report to his employer, leave the workplace and consult a doctor immediately, even if symptoms may appear mild. Employers must track and record these cases as part of Safe Management Measures. For incapacitated or unconscious individuals, employers must clear the area of other personnel and administer aid immediately. Employers should call 995 for an emergency ambulance to ferry them to the nearest hospital. Manage confirmed cases: A follow-up plan must be put in place in the event of a confirmed case. Upon being notified of a confirmed case, employers must adopt the following precautionary measures: Immediately vacate and cordon-off the immediate section of the workplace premises where the confirmed case worked.', 'score': 0.6463999999999714, 'probability': 0.8231999999999857, 'question': None, 'meta': {'end_date': '2021-09-30T00:00:00', '_split_id': 24, 'category': None, 'start_date': '2021-08-19T00:0

# get_summaries()
- return the Covid 19 Regulation Summaries, given a list of categories

In [ ]:
def get_summaries(categories:list):
    today = date.today()
    date_range = list(pd.date_range(start=today,periods=100))
    return datastore.datastore.get_all_documents(config['index'],filters = {"category":categories,"end_date":date_range}) 

In [ ]:
get_summaries(["sports"])

In [ ]:
datastore.datastore.get_all_documents()